In [1]:
import torch
import torch.nn as nn

In [2]:
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

    def forward(self, img):
        feature = self.conv(img)
        # print(feature.shape)
        output = self.fc(feature.view(img.shape[0], -1))  # flatten img to batch * 1-dim vector

        return output

In [20]:
net = LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [21]:
batch = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch)

In [6]:
for X, y in train_iter:
    print(X.shape)
    break

torch.Size([256, 1, 28, 28])


In [18]:
loss_fn = nn.CrossEntropyLoss()
lr, num_epochs = 0.001, 5
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [8]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('../runs/lenet')

In [19]:
d2l.train_func(net, train_iter, test_iter, loss_fn, num_epochs, batch, device, lr=lr, optimizer=optim, writer=writer)

training on  cuda
epoch 1, loss 0.0071, train acc 0.338, test acc 0.597, time 4.7 sec
epoch 2, loss 0.0037, train acc 0.626, test acc 0.679, time 4.6 sec
epoch 3, loss 0.0030, train acc 0.717, test acc 0.738, time 4.7 sec
epoch 4, loss 0.0027, train acc 0.743, test acc 0.749, time 5.3 sec
epoch 5, loss 0.0025, train acc 0.756, test acc 0.758, time 4.6 sec


In [22]:
loss_fn = nn.CrossEntropyLoss()
lr, num_epochs = 0.001, 50

optim = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=0.01)
d2l.train_func(net, train_iter, test_iter, loss_fn, num_epochs, batch, device, lr=lr, optimizer=optim, writer=writer)

training on  cuda
epoch 1, loss 0.0090, train acc 0.101, test acc 0.100, time 4.7 sec
epoch 2, loss 0.0090, train acc 0.098, test acc 0.100, time 4.6 sec
epoch 3, loss 0.0090, train acc 0.100, test acc 0.100, time 4.6 sec
epoch 4, loss 0.0090, train acc 0.097, test acc 0.100, time 4.5 sec
epoch 5, loss 0.0090, train acc 0.099, test acc 0.100, time 5.2 sec
epoch 6, loss 0.0090, train acc 0.102, test acc 0.100, time 4.7 sec
epoch 7, loss 0.0090, train acc 0.100, test acc 0.100, time 4.9 sec
epoch 8, loss 0.0090, train acc 0.099, test acc 0.100, time 4.7 sec


KeyboardInterrupt: 

In [10]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('../runs/lenet')

In [17]:
images, labels = iter(train_iter).next()

In [18]:
writer.add_graph(net, images)
writer.close()

In [10]:
!tensorboard --logdir=../runs

^C


TypeError: 'NoneType' object is not iterable